task name: Implement Matrix Factorization with Options
task description: Implement the provided C# code as a dotnet interactive notebook, saving the output to a file.

In [ ]:
#r "nuget:Microsoft.ML.Recommender"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
private const uint MatrixColumnCount = 60;
private const uint MatrixRowCount = 100;

private class MatrixElement
{
    [KeyType(MatrixColumnCount)]
    public uint MatrixColumnIndex { get; set; }

    [KeyType(MatrixRowCount)]
    public uint MatrixRowIndex { get; set; }

    public float Value { get; set; }

    public float Score { get; set; }
}

private static List<MatrixElement> GenerateMatrix()
{
    var dataMatrix = new List<MatrixElement>();
    for (uint i = 0; i < MatrixColumnCount; ++i)
        for (uint j = 0; j < MatrixRowCount; ++j)
            dataMatrix.Add(new MatrixElement()
            {
                MatrixColumnIndex = i,
                MatrixRowIndex = j,
                Value = (i + j) % 5
            });

    return dataMatrix;
}

var dataPoints = GenerateMatrix();
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

In [ ]:
var options = new MatrixFactorizationTrainer.Options
{
    MatrixColumnIndexColumnName = nameof(MatrixElement.MatrixColumnIndex),
    MatrixRowIndexColumnName = nameof(MatrixElement.MatrixRowIndex),
    LabelColumnName = nameof(MatrixElement.Value),
    NumberOfIterations = 10,
    NumberOfThreads = 1,
    ApproximationRank = 32,
    LearningRate = 0.3
};

var pipeline = mlContext.Recommendation().Trainers.MatrixFactorization(options);
var model = pipeline.Fit(trainingData);

In [ ]:
var transformedData = model.Transform(trainingData);
var predictions = mlContext.Data.CreateEnumerable<MatrixElement>(transformedData, reuseRowObject: false).Take(5).ToList();
foreach (var p in predictions)
    Console.WriteLine($"Actual value: {p.Value:F3}, Predicted score: {p.Score:F3}");

var metrics = mlContext.Regression.Evaluate(transformedData, labelColumnName: nameof(MatrixElement.Value), scoreColumnName: nameof(MatrixElement.Score));

Console.WriteLine("Mean Absolute Error: " + metrics.MeanAbsoluteError);
Console.WriteLine("Mean Squared Error: " + metrics.MeanSquaredError);
Console.WriteLine("Root Mean Squared Error: " + metrics.RootMeanSquaredError);
Console.WriteLine("RSquared: " + metrics.RSquared);